In [21]:
!pip install streamlit pyngrok PyJWT bcrypt --quiet

In [22]:
%%writefile app.py
import streamlit as st
import sqlite3
import re
import jwt
import datetime
import bcrypt

SECRET_KEY = "policynav_secret_key"

def load_css():
    st.markdown("""
    <style>
    @import url('https://fonts.googleapis.com/css2?family=DM+Sans:wght@300;400;500;600&family=DM+Serif+Display:ital@0;1&display=swap');

    * { box-sizing: border-box; }

    .stApp {
        background-color: #F7F6F3;
        font-family: 'DM Sans', sans-serif;
    }

    #MainMenu, footer, header { visibility: hidden; }
    .block-container { padding-top: 3rem !important; }

    .wordmark {
        font-family: 'DM Serif Display', serif;
        font-size: 1.1rem;
        letter-spacing: 0.02em;
        color: #111;
        position: fixed;
        top: 2rem;
        left: 2.5rem;
        z-index: 100;
    }
    .wordmark span { font-style: italic; color: #888; }

    .hero {
        text-align: center;
        padding: 5rem 0 3rem 0;
    }
    .hero-label {
        font-size: 0.7rem;
        font-weight: 500;
        letter-spacing: 0.18em;
        text-transform: uppercase;
        color: #999;
        margin-bottom: 1.2rem;
    }
    .hero-title {
        font-family: 'DM Serif Display', serif;
        font-size: 3.8rem;
        color: #111;
        line-height: 1.1;
        margin: 0 0 1.2rem 0;
    }
    .hero-title em { font-style: italic; color: #777; }
    .hero-sub { font-size: 0.95rem; color: #888; font-weight: 400; }

    .card {
        background: #FFFFFF;
        border: 1px solid #E8E6E1;
        border-radius: 16px;
        padding: 2.5rem 2.5rem 1rem 2.5rem;
        max-width: 420px;
        margin: 0 auto 1.5rem auto;
        box-shadow: 0 2px 20px rgba(0,0,0,0.04);
    }
    .card-title {
        font-family: 'DM Serif Display', serif;
        font-size: 1.6rem;
        color: #111;
        margin-bottom: 0.3rem;
    }
    .card-sub { font-size: 0.82rem; color: #AAA; font-weight: 400; }

    .stTextInput label, .stSelectbox label {
        font-size: 0.75rem !important;
        font-weight: 500 !important;
        letter-spacing: 0.08em !important;
        text-transform: uppercase !important;
        color: #888 !important;
    }
    .stTextInput > div > div > input {
        background: #FAFAF8 !important;
        border: 1px solid #E0DED9 !important;
        border-radius: 10px !important;
        padding: 0.7rem 1rem !important;
        font-size: 0.95rem !important;
        color: #111 !important;
        font-family: 'DM Sans', sans-serif !important;
        transition: border-color 0.2s !important;
    }
    .stTextInput > div > div > input:focus {
        border-color: #111 !important;
        background: #fff !important;
        box-shadow: none !important;
    }
    .stTextInput input::placeholder { color: #C8C5BF !important; }
    .stTextInput input { -webkit-text-fill-color: #111 !important; }

    div[data-baseweb="select"] > div {
        background: #FAFAF8 !important;
        border: 1px solid #E0DED9 !important;
        border-radius: 10px !important;
        color: #111 !important;
    }

    .stButton > button {
        width: 100%;
        background: #111 !important;
        color: #fff !important;
        border: none !important;
        padding: 0.72rem 1.5rem !important;
        border-radius: 10px !important;
        font-size: 0.88rem !important;
        font-weight: 500 !important;
        font-family: 'DM Sans', sans-serif !important;
        letter-spacing: 0.03em !important;
        transition: background 0.2s, transform 0.1s !important;
        margin-bottom: 0.4rem !important;
    }
    .stButton > button:hover {
        background: #2a2a2a !important;
        transform: translateY(-1px) !important;
    }

    .stSuccess > div {
        background: #F0FAF4 !important;
        border: 1px solid #B7E5C8 !important;
        border-radius: 10px !important;
        color: #1D6B3A !important;
        font-size: 0.88rem !important;
    }
    .stError > div {
        background: #FEF2F2 !important;
        border: 1px solid #FECACA !important;
        border-radius: 10px !important;
        color: #991B1B !important;
        font-size: 0.88rem !important;
    }
    .stInfo > div {
        background: #F8F8F6 !important;
        border: 1px solid #E0DED9 !important;
        border-radius: 10px !important;
        color: #555 !important;
        font-size: 0.88rem !important;
    }

    .dash-header {
        background: #111;
        color: #fff;
        border-radius: 14px;
        padding: 2rem 2.5rem;
        margin-bottom: 1.5rem;
    }
    .dash-header .label {
        font-size: 0.7rem;
        letter-spacing: 0.15em;
        text-transform: uppercase;
        color: #666;
        margin-bottom: 0.5rem;
    }
    .dash-header .name {
        font-family: 'DM Serif Display', serif;
        font-size: 2rem;
        margin: 0 0 0.3rem 0;
    }
    .dash-header .email { font-size: 0.85rem; color: #777; }
    </style>
    """, unsafe_allow_html=True)

def init_db():
    conn = sqlite3.connect("users.db", check_same_thread=False)
    c = conn.cursor()
    c.execute("""CREATE TABLE IF NOT EXISTS users (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        username TEXT NOT NULL,
        email TEXT UNIQUE NOT NULL,
        password BLOB NOT NULL,
        security_q TEXT NOT NULL,
        security_a BLOB NOT NULL
    )""")
    conn.commit()
    conn.close()

def validate_email(e): return re.match(r"[^@]+@[^@]+\.[a-zA-Z]{2,}", e)
def validate_password(p): return p.isalnum() and len(p) >= 8
def hash_text(t): return bcrypt.hashpw(t.encode(), bcrypt.gensalt())
def verify_text(t, h): return bcrypt.checkpw(t.encode(), h)

def create_token(email, username):
    return jwt.encode({"email": email, "username": username,
        "exp": datetime.datetime.utcnow() + datetime.timedelta(minutes=30)},
        SECRET_KEY, algorithm="HS256")

def verify_token(token):
    try: return jwt.decode(token, SECRET_KEY, algorithms=["HS256"])
    except: return None

for k, v in [("page","home"),("reset_email",None),("question_shown",False),("answer_verified",False)]:
    if k not in st.session_state: st.session_state[k] = v

def main():
    st.set_page_config(page_title="PolicyNav", page_icon="◈", layout="centered")
    load_css()
    init_db()

    st.markdown('<div class="wordmark">Policy<span>Nav</span></div>', unsafe_allow_html=True)

    # HOME
    if st.session_state.page == "home":
        st.markdown("""
        <div class="hero">
            <div class="hero-label">Secure Authentication</div>
            <div class="hero-title">Navigate with<br><em>confidence.</em></div>
            <div class="hero-sub">Simple, secure access to your policy dashboard.</div>
        </div>
        """, unsafe_allow_html=True)
        col1, col2, col3 = st.columns([1,2,1])
        with col2:
            if st.button("Sign in", key="home_login"): st.session_state.page="login"; st.rerun()
            if st.button("Create account", key="home_signup"): st.session_state.page="signup"; st.rerun()
            if st.button("Forgot password", key="home_forgot"): st.session_state.page="forgot"; st.rerun()

    # LOGIN
    elif st.session_state.page == "login":
        col1, col2, col3 = st.columns([1,2,1])
        with col2:
            st.markdown('<div class="card"><div class="card-title">Welcome back</div><div class="card-sub">Sign in to your account</div></div>', unsafe_allow_html=True)
            email = st.text_input("Email", placeholder="you@example.com", key="li_email")
            password = st.text_input("Password", type="password", placeholder="••••••••", key="li_pass")
            st.markdown("<br>", unsafe_allow_html=True)
            if st.button("Sign in →", key="li_btn"):
                if not email or not password: st.error("Please fill in all fields.")
                else:
                    conn = sqlite3.connect("users.db")
                    c = conn.cursor()
                    c.execute("SELECT username, password FROM users WHERE email=?", (email,))
                    user = c.fetchone(); conn.close()
                    if user and verify_text(password, user[1]):
                        st.session_state.token = create_token(email, user[0])
                        st.session_state.page = "dashboard"; st.rerun()
                    else: st.error("Invalid email or password.")
            if st.button("← Back", key="li_back"): st.session_state.page="home"; st.rerun()

    # SIGNUP
    elif st.session_state.page == "signup":
        col1, col2, col3 = st.columns([1,2,1])
        with col2:
            st.markdown('<div class="card"><div class="card-title">Create account</div><div class="card-sub">Join PolicyNav today</div></div>', unsafe_allow_html=True)
            username  = st.text_input("Username", placeholder="yourname", key="su_user")
            email     = st.text_input("Email", placeholder="you@example.com", key="su_email")
            password  = st.text_input("Password", type="password", placeholder="Min 8 chars, alphanumeric", key="su_pass")
            confirm   = st.text_input("Confirm password", type="password", placeholder="Re-enter password", key="su_conf")
            security_q = st.selectbox("Security question", [
                "What is your pet name?","What is your favorite color?",
                "What is your favorite book?","What is your favorite movie?",
                "Who is your favorite teacher?"], key="su_sq")
            security_a = st.text_input("Answer", placeholder="No spaces", key="su_sa")
            st.markdown("<br>", unsafe_allow_html=True)
            if st.button("Create account →", key="su_btn"):
                if not all([username,email,password,confirm,security_a]): st.error("All fields are required.")
                elif not validate_email(email): st.error("Invalid email format.")
                elif not validate_password(password): st.error("Password must be alphanumeric and 8+ characters.")
                elif password != confirm: st.error("Passwords do not match.")
                elif " " in password or " " in security_a: st.error("No spaces allowed.")
                else:
                    conn = sqlite3.connect("users.db"); c = conn.cursor()
                    try:
                        c.execute("INSERT INTO users (username,email,password,security_q,security_a) VALUES (?,?,?,?,?)",
                            (username, email, hash_text(password), security_q, hash_text(security_a.lower().strip())))
                        conn.commit(); st.success("Account created — please sign in.")
                    except sqlite3.IntegrityError: st.error("Email already in use.")
                    conn.close()
            if st.button("← Back", key="su_back"): st.session_state.page="home"; st.rerun()

    # FORGOT
    elif st.session_state.page == "forgot":
        col1, col2, col3 = st.columns([1,2,1])
        with col2:
            st.markdown('<div class="card"><div class="card-title">Reset password</div><div class="card-sub">Answer your security question</div></div>', unsafe_allow_html=True)
            email = st.text_input("Registered email", placeholder="you@example.com", key="fp_email")
            if st.button("Continue →", key="fp_check"):
                if not email: st.error("Please enter your email.")
                else:
                    conn = sqlite3.connect("users.db"); c = conn.cursor()
                    c.execute("SELECT security_q FROM users WHERE email=?", (email,))
                    user = c.fetchone(); conn.close()
                    if user:
                        st.session_state.reset_email = email
                        st.session_state.question_shown = True
                        st.success(f"Question: {user[0]}")
                    else: st.error("Email not found.")
            if st.session_state.question_shown:
                answer = st.text_input("Your answer", placeholder="No spaces", key="fp_ans")
                if st.button("Verify →", key="fp_verify"):
                    conn = sqlite3.connect("users.db"); c = conn.cursor()
                    c.execute("SELECT security_a FROM users WHERE email=?", (st.session_state.reset_email,))
                    stored = c.fetchone(); conn.close()
                    if stored and verify_text(answer.lower().strip(), stored[0]):
                        st.session_state.answer_verified = True
                        st.success("Verified. Set your new password below.")
                    else: st.error("Incorrect answer.")
            if st.session_state.answer_verified:
                new_pass = st.text_input("New password", type="password", key="fp_new")
                conf_pass = st.text_input("Confirm new password", type="password", key="fp_conf")
                if st.button("Reset password →", key="fp_reset"):
                    if new_pass != conf_pass: st.error("Passwords do not match.")
                    elif not validate_password(new_pass): st.error("Password must be alphanumeric and 8+ characters.")
                    elif " " in new_pass: st.error("No spaces allowed.")
                    else:
                        conn = sqlite3.connect("users.db"); c = conn.cursor()
                        c.execute("UPDATE users SET password=? WHERE email=?", (hash_text(new_pass), st.session_state.reset_email))
                        conn.commit(); conn.close()
                        st.success("Password reset. Please sign in.")
                        st.session_state.question_shown = False
                        st.session_state.answer_verified = False
                        st.session_state.reset_email = None
            if st.button("← Back", key="fp_back"):
                st.session_state.page = "home"
                st.session_state.question_shown = False
                st.session_state.answer_verified = False
                st.rerun()

    # DASHBOARD
    elif st.session_state.page == "dashboard":
        payload = verify_token(st.session_state.get("token"))
        if payload:
            col1, col2, col3 = st.columns([1,2,1])
            with col2:
                st.markdown(f"""
                <div class="dash-header">
                    <div class="label">Signed in as</div>
                    <div class="name">{payload['username']}</div>
                    <div class="email">{payload['email']}</div>
                </div>
                """, unsafe_allow_html=True)
                st.info("Your dashboard is ready. Features coming soon.")
                st.markdown("<br>", unsafe_allow_html=True)
                if st.button("Sign out", key="logout_btn"):
                    st.session_state.clear(); st.session_state.page="home"; st.rerun()
        else:
            st.session_state.page = "login"; st.rerun()

if __name__ == "__main__":
    main()

Overwriting app.py


In [23]:
import os
from pyngrok import ngrok

ngrok.kill()
os.system("pkill -9 ngrok")

256

In [24]:
from pyngrok import ngrok
import subprocess, time, os

NGROK_AUTH_TOKEN = "39yGsZsEsKXf5vNuCWmZ8gTZG3P_2GsMzXTvxZqUAT2uHWG7P"

os.system("pkill -9 streamlit")
os.system("pkill -9 ngrok")
ngrok.kill()
time.sleep(5)

ngrok.set_auth_token(NGROK_AUTH_TOKEN)

process = subprocess.Popen(
    ["streamlit", "run", "app.py", "--server.port", "8501", "--server.headless", "true"],
    stdout=subprocess.DEVNULL,
    stderr=subprocess.DEVNULL
)
time.sleep(8)

public_url = ngrok.connect(8501)
print(f"🌍 URL: {public_url}")

🌍 URL: NgrokTunnel: "https://chubby-vania-unwonderfully.ngrok-free.dev" -> "http://localhost:8501"


In [25]:
import sqlite3
import pandas as pd

conn = sqlite3.connect('users.db')
df = pd.read_sql_query("SELECT id, username, email, security_q FROM users", conn)
conn.close()
print(df)

   id username                        email              security_q
0   1      abc  kuruvalaramya2006@gmail.com  What is your pet name?
